In [ ]:
# default_exp engine

# Engine

> Use a chess engine to analyze moves.

In [ ]:
#export
import chess.engine
import chess.pgn
import pandas as pd

In [ ]:
stockfish = "stockfish_14_linux_x64_popcnt/stockfish_14_x64_popcnt" # lcation of stockfish executable or alias

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci(stockfish)

In [ ]:
pgn = open("data/magnus.pgn")

In [ ]:
game = chess.pgn.read_game(pgn)

In [ ]:
game.headers

Headers(Event='Rated Blitz game', Site='https://lichess.org/lAV0T0zl', Date='2021.12.23', Round='?', White='DrNykterstein', Black='may6enexttime', Result='1-0', BlackElo='2974', BlackRatingDiff='-2', BlackTitle='GM', ECO='B20', Termination='Normal', TimeControl='180+0', UTCDate='2021.12.23', UTCTime='23:28:07', Variant='Standard', WhiteElo='3212', WhiteRatingDiff='+2', WhiteTitle='GM')

In [ ]:
#export
def evaluate_game(game, engine, limit=chess.engine.Limit(time=.1)):
    """Go through a game move-by-move and evaluate each position with an engine.
    """
    board = game.board()
    moves = game.mainline_moves()
    evals = []
    for move in moves:
        board.push(move)
        info = engine.analyse(board, limit=limit)
        score = info["score"].pov(chess.WHITE).score(mate_score=10_000)
        evals.append(score)
    return evals

In [ ]:
evals = evaluate_game(game, engine)

In [ ]:
evals

[47,
 21,
 -27,
 -30,
 -24,
 -2,
 17,
 50,
 76,
 109,
 -97,
 -62,
 -71,
 -98,
 -102,
 -90,
 -147,
 -122,
 -146,
 -161,
 -167,
 -166,
 -165,
 -49,
 -58,
 -79,
 -41,
 -72,
 -61,
 -80,
 -65,
 -7,
 -45,
 4,
 -1,
 16,
 1,
 3,
 8,
 290,
 289,
 362,
 437,
 350,
 397,
 354,
 382,
 551,
 543,
 537,
 517,
 513,
 530,
 591,
 596]

In [ ]:
#export
def evaluate_pgn(pgn, engine, limit=chess.engine.Limit(time=.1)):
    """Evaluate all games in a PGN with the engine.
    """
    game = chess.pgn.read_game(pgn)
    all_evals = []
    while game:
        evals = evaluate_game(game, engine, limit)
        all_evals.append(evals)
        game = chess.pgn.read_game(pgn)
    df = pd.DataFrame(all_evals)
    return df

In [ ]:
pgn = open("data/magnus.pgn")

In [ ]:
evals = evaluate_pgn(pgn, engine)

55
120
137
74
53


In [ ]:
evals.head()

,0,1,2,3,4,5,6,7,8,9,...,127,128,129,130,131,132,133,134,135,136
0,52,55,-27,-15,0,22,-5,55,28,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,52,37,34,34,21,31,15,24,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38,44,-11,0,-1,-13,-16,1,5,5,...,9986.0,9984.0,9991.0,9992.0,9993.0,9994.0,9994.0,5870.0,9987.0,9988.0
3,40,31,37,35,30,21,16,12,21,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27,40,-29,-25,-14,-13,-4,-17,-29,-37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
evals.to_csv("data/magnus_evals.csv")